<a href="https://colab.research.google.com/github/sak1b0/proteiNN/blob/master/run_forest_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt

url_train='https://raw.githubusercontent.com/sak1b0/proteiNN/master/train_formatted.csv'
url_test='https://raw.githubusercontent.com/sak1b0/proteiNN/master/test_formatted.csv'

df_train=pd.read_csv(url_train,header=None)
df_test=pd.read_csv(url_test,header=None)



df_train=np.asarray(df_train)
df_test=np.asarray(df_test)

X_train = df_train[:, 0]
y_train = df_train[:, 1]

X_test = df_test[:, 0]
y_test = df_test[:, 1]

def debug_me():
  print('train dataframe: ',df_train.shape)
  print('X train shape: ',X_train.shape)
  print('y train shape: ',y_train.shape)

  print('test dataframe: ',df_test.shape)
  print('X test shape: ',X_test.shape)
  print('y test shape: ',y_test.shape)
  

debug_me()

max_length_train=0
for item in range (len(X_train)):
  max_length_train=max(max_length_train,len(X_train[item])) #finding maximum length of string in training
  
  
print('Maximum length of train: ',max_length_train)

max_length_test=0
for item in range (len(X_test)):
  max_length_test=max(max_length_test,len(X_test[item])) #finding maximum length of string in testing set
  
print('Maximum length of test: ',max_length_test)

#max_length_train=max(max_length_train,max_length_test)



for item in range (len(X_train)):
  X_train[item] = X_train[item]+'Z'*(max_length_train-len(X_train[item])) # padding with 'Z' to make all of them the same length
  #X_train[item] = one_hot.fit_transform(X_train[item])
  
  
for item in range (len(y_train)):
  y_train[item] = y_train[item]+'Z'*(max_length_train-len(y_train[item])) # padding with 'Z' to make all of them the same length
  #y_train[item] = one_hot.fit_transform(y_train[item])
  
  
for item in range (len(X_test)):
  X_test[item] = X_test[item]+'Z'*(max_length_train-len(X_test[item])) # padding with 'Z' to make all of them the same length
  #X_test[item] =one_hot.fit_transform(X_test[item])
  
  
for item in range (len(y_test)):
  y_test[item] = y_test[item]+'Z'*(max_length_train-len(y_test[item])) # padding with 'Z' to make all of them the same length  
  #y_test[item] = one_hot.fit_transform(y_test[item])
  
  
#print(X_train[0])
#print(y_train[0])


print('\nafter padding: \n')

debug_me()
#print(X_train[0])
#print(y_train[0])

#====================one cold encoder start=======

def OneColdEncoder(x):
  
  ret = list()
  
  encode = [[0 for i in range(26)] for j in range(26)]
  
  for i in range(26):
    encode[i][i] = 1
  
  ret = encode[ord(x)-65]
    
  return ret


#====================one cold encoder end=========


#encoding to integer of X_train

s = list(X_train)

for item in range(len(s)):
  s[item]=list(s[item])
  
print(s)

k = []

for i in range(len(s)):
  t = []
  for item in range(len(s[i])):
    t.insert(2000, OneColdEncoder(s[i][item]))
    s[i][item]=ord(s[i][item])-64
  k.append(t)


X_train = np.array(k).reshape(4590,1303,26,1)

#encoding to integer of y_train

s = list(y_train)

for item in range(len(s)):
  s[item]=list(s[item])
  
print(s)

k = []

for i in range(len(s)):
  t = []
  for item in range(len(s[i])):
    t.insert(2000, OneColdEncoder(s[i][item]))
    s[i][item]=ord(s[i][item])-64
  k.append(t)


y_train = np.array(k).reshape(4590,1303,26,1)

#encoding to integer of X_test

s = list(X_test)

for item in range(len(s)):
  s[item]=list(s[item])
  
print(s)

k = []

for i in range(len(s)):
  t = []
  for item in range(len(s[i])):
    t.insert(2000, OneColdEncoder(s[i][item]))
    s[i][item]=ord(s[i][item])-64
  k.append(t)


X_test = np.array(k).reshape(1199,1303,26,1)



#encoding to integer of y_test

s = list(y_test)

for item in range(len(s)):
  s[item]=list(s[item])
  
print(s)

k = []

for i in range(len(s)):
  t = []
  for item in range(len(s[i])):
    t.insert(2000, OneColdEncoder(s[i][item]))
    s[i][item]=ord(s[i][item])-64
  k.append(t)


y_test = np.array(k).reshape(1199,1303,26,1)




print('\nafter encoding to integers\n')
debug_me()

display(X_train[0])

#=======================this is where the fun part begins====================

model=Sequential()

model.add(LSTM((1303),batch_input_shape=(None,1,1303),return_sequences=True))

model.compile(loss='mean_absolute_error',optimizer='adam',metrics=['accuracy'])

model.summary()

print(model.input_shape)
print(model.output_shape)

history=model.fit(X_train,y_train,epochs=50,validation_data=(X_test,y_test))



